# COS利用のための設定

In [1]:
import ibm_boto3
from ibm_botocore.client import Config
import os
import json
import warnings
import time

In [ ]:
# 個別に設定して下さい
cos_credentials = {
  "apikey": "xxxxx",
  "cos_hmac_keys": {
    "access_key_id": "xxxxx",
    "secret_access_key": "xxxxx"
  },
  "endpoints": "https://cos-service.bluemix.net/endpoints",
  "iam_apikey_description": "Auto generated apikey during resource-key operation for Instance - crn:v1:bluemix:public:cloud-object-storage:global:a/0960d86217f3b669ae4408f3b4deaa08:38bc5a04-c294-44d0-93d3-57a963ea4f0d::",
  "iam_apikey_name": "xxxxx",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "xxxxx",
  "resource_instance_id": "xxxxx"
}

In [3]:
api_key = cos_credentials['apikey']
service_instance_id = cos_credentials['resource_instance_id']
auth_endpoint = 'https://iam.bluemix.net/oidc/token'
service_endpoint = 'https://s3-api.us-geo.objectstorage.softlayer.net'

In [4]:
cos = ibm_boto3.resource('s3',
                         ibm_api_key_id=api_key,
                         ibm_service_instance_id=service_instance_id,
                         ibm_auth_endpoint=auth_endpoint,
                         config=Config(signature_version='oauth'),
                         endpoint_url=service_endpoint)

In [ ]:
# 入力backetと出力backetの定義
# 個別に行って下さい
buckets = ['xxxxx', 'yyyyy']

In [6]:
for bucket in buckets:
    if not cos.Bucket(bucket) in cos.buckets.all():
        print('Creating bucket "{}"...'.format(bucket))
        try:
            cos.create_bucket(Bucket=bucket)
        except ibm_boto3.exceptions.ibm_botocore.client.ClientError as e:
            print('Error: {}.'.format(e.response['Error']['Message']))

In [7]:
print(list(cos.buckets.all()))

[s3.Bucket(name='20180328project0d195723b6474adbbe32a88e80167381'), s3.Bucket(name='bucket-aka-1'), s3.Bucket(name='dlaas-input-aka1'), s3.Bucket(name='dlaas-output-aka1'), s3.Bucket(name='dltest1860e226fce334f69b71ebf5726a66f81'), s3.Bucket(name='input-backet-aka2'), s3.Bucket(name='testvrb4a94a00382c471b948ca35468df71a0'), s3.Bucket(name='vrproject7ac8b023068546a08745a4abd939c5aa'), s3.Bucket(name='watsonstudio20180413-donotdelete-pr-zejryh21crj12z'), s3.Bucket(name='xxx-donotdelete-pr-ktftsnknk5rqbj')]


# mnintデータをダウンロードしCOSに保存します

In [ ]:
!pip install wget

In [8]:
link = 'https://s3.amazonaws.com/img-datasets/mnist.npz'
import wget, os

data_dir = 'MNIST_KERAS_DATA'
if not os.path.isdir(data_dir):
    os.mkdir(data_dir)

if not os.path.isfile(os.path.join(data_dir, os.path.join(link.split('/')[-1]))):
    wget.download(link, out=data_dir)  
        
!ls MNIST_KERAS_DATA

mnist.npz


In [9]:
# データファイルを入力bucketにアップロードします
bucket_name = buckets[0]
bucket_obj = cos.Bucket(bucket_name)

for filename in os.listdir(data_dir):
    with open(os.path.join(data_dir, filename), 'rb') as data: 
        bucket_obj.upload_file(os.path.join(data_dir, filename), filename)
        print('{} is uploaded.'.format(filename))
        
# アップロードしたファイルの確認
for obj in bucket_obj.objects.all():
    print('Object key: {}'.format(obj.key))
    print('Object size (kb): {}'.format(obj.size/1024))

mnist.npz is uploaded.
Object key: cifar-10-tf-test.pkl
Object size (kb): 30078.3701171875
Object key: cifar-10-tf-train.pkl
Object size (kb): 120039.2705078125
Object key: cifar-10-tf-valid.pkl
Object size (kb): 30009.9736328125
Object key: mnist-tf-test.pkl
Object size (kb): 7666.21875
Object key: mnist-tf-train.pkl
Object size (kb): 38330.28125
Object key: mnist-tf-valid.pkl
Object size (kb): 7666.21875
Object key: mnist.npz
Object size (kb): 11221.126953125


# Watson Machine Learning管理クライアントでの作業

In [ ]:
!rm -rf $PIP_BUILD/watson-machine-learning-client
!pip install --upgrade watson-machine-learning-client

In [10]:
import urllib3, requests, json, base64, time, os
warnings.filterwarnings('ignore')

In [ ]:
# Watson Machine LearningのCredntial情報
# 個別に設定して下さい。
wml_credentials = {
  "url": "https://ibm-watson-ml.mybluemix.net",
  "username": "xxxxx",
  "password": "xxxxx",
  "instance_id": "xxxxx"
}

In [12]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient
client = WatsonMachineLearningAPIClient(wml_credentials)
print(client.version)

1.0.109


2018-04-27 11:53:06,564 - watson_machine_learning_client.metanames - WARNING - 'AUTHOR_EMAIL' meta prop is deprecated. It will be ignored.
2018-04-27 11:53:31,493 - watson_machine_learning_client.metanames - WARNING - 'AUTHOR_EMAIL' meta prop is deprecated. It will be ignored.


# Trainingの定義

In [13]:
# Authorの定義　個別に行って下さい
author_email = 'xxx@ibm.com'

In [14]:
model_definition_metadata = {
            client.repository.DefinitionMetaNames.NAME: "MNIST-MLP",
            client.repository.DefinitionMetaNames.AUTHOR_EMAIL: author_email,
            client.repository.DefinitionMetaNames.FRAMEWORK_NAME: "tensorflow",
            client.repository.DefinitionMetaNames.FRAMEWORK_VERSION: "1.5",
            client.repository.DefinitionMetaNames.RUNTIME_NAME: "python",
            client.repository.DefinitionMetaNames.RUNTIME_VERSION: "3.5",
            client.repository.DefinitionMetaNames.EXECUTION_COMMAND: "python3 mnist_mlp.py"
            }

In [15]:
# モデル定義 (Python/Kerasのコード)のダウンロード
# MNIST.zipにはmnist-nlp.pyとmninst-cnn.pyの2つのコードが含まれていますが、以下のサンプルでは
# mnist-nlp.pyのみを利用します

!rm -rf MNIST.zip

filename_mnist = 'MNIST.zip'

if os.path.isfile(filename_mnist):
    !ls 'MNIST.zip'
else:
    !wget https://github.com/pmservice/wml-sample-models/raw/master/keras/mnist/MNIST.zip
    !ls 'MNIST.zip'

--2018-04-27 11:53:02--  https://github.com/pmservice/wml-sample-models/raw/master/keras/mnist/MNIST.zip
Resolving github.com (github.com)... 192.30.253.112, 192.30.253.113
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/pmservice/wml-sample-models/master/keras/mnist/MNIST.zip [following]
--2018-04-27 11:53:02--  https://raw.githubusercontent.com/pmservice/wml-sample-models/master/keras/mnist/MNIST.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3836 (3.7K) [application/zip]
Saving to: ‘MNIST.zip’

100%[======================================>] 3,836       --.-K/s   in 0s      

2018-04-27 11:53:02 (39.8 MB/s) - ‘MNIST.zip’ saved [3836/3836]

MNIST.zip


In [16]:
# リポジトリに定義ファイルの登録
definition_details = client.repository.store_definition(filename_mnist, model_definition_metadata)

definition_url = client.repository.get_definition_url(definition_details)
definition_uid = client.repository.get_definition_uid(definition_details)
print(definition_url)

https://ibm-watson-ml.mybluemix.net/v3/ml_assets/training_definitions/8d712ccf-9f7c-4176-bebd-a7a60005b1fc


In [17]:
# 定義ファイルの一覧
client.repository.list_definitions()

------------------------------------  -------------------------------------------------  ------------------------  ----------
GUID                                  NAME                                               CREATED                   FRAMEWORK
90e0bd84-88bc-4f1f-8198-e6e3852983b2  TENT pred                                          2018-04-13T07:15:43.505Z  wml
10f36b94-4c64-46f2-845d-48e6c2f73dc5  Tent-pred                                          2018-04-13T09:22:19.699Z  wml
d6a3ce6c-27e3-45bc-8c17-8e909c5b72c0  test-pred300                                       2018-04-13T09:26:06.992Z  wml
bc17f865-c385-4f8b-a015-37b5074714f7  MNIST v1                                           2018-04-13T11:06:47.185Z  tensorflow
5a05d09d-ff8d-450b-a72c-262dadd3a52c  CIFAR10 v1                                         2018-04-13T11:09:52.326Z  tensorflow
51ff1d51-b8bb-4883-a9bd-09e504e9e1c7  MNIST v2                                           2018-04-16T00:32:31.568Z  tensorflow
3ba0d4f4-88cf-

# Experimentの定義

In [18]:
# サポートしている項目の一覧表示
client.repository.ExperimentMetaNames.show()

--------------------------  ----  --------
META_PROP NAME              TYPE  REQUIRED
NAME                        str   Y
TAGS                        list  N
DESCRIPTION                 str   N
AUTHOR_NAME                 str   N
EVALUATION_METHOD           str   N
EVALUATION_METRICS          list  N
TRAINING_REFERENCES         list  Y
TRAINING_DATA_REFERENCE     dict  Y
TRAINING_RESULTS_REFERENCE  dict  Y
--------------------------  ----  --------


In [19]:
# 学習時の入力用bucketの定義
TRAINING_DATA_REFERENCE = {
                            "connection": {
                                "endpoint_url": service_endpoint,
                                "aws_access_key_id": cos_credentials['cos_hmac_keys']['access_key_id'],
                                "aws_secret_access_key": cos_credentials['cos_hmac_keys']['secret_access_key']
                            },
                            "source": {
                                "bucket": buckets[0],
                            },
                            "type": "s3"
}

In [20]:
# 学習時の出力用bucketの定義
TRAINING_RESULTS_REFERENCE = {
                                "connection": {
                                    "endpoint_url": service_endpoint,
                                    "aws_access_key_id": cos_credentials['cos_hmac_keys']['access_key_id'],
                                    "aws_secret_access_key": cos_credentials['cos_hmac_keys']['secret_access_key']
                                },
                                "target": {
                                    "bucket": buckets[1],
                                },
                                "type": "s3"
}

#### HPOパラメータの指定

**val_acc**(検証データの認識率)を最大にしたいので、これをOptimizerのObjectiveにしています。  
**num_optimizer_steps**で、どの程度細かくパラメータを変化させるかを指定します。

In [21]:
HPO = {
        "method": {
            "name": "rbfopt",
            "parameters": [
                client.experiments.HPOMethodParam("objective", "val_acc"),
                client.experiments.HPOMethodParam("maximize_or_minimize", "maximize"),
                client.experiments.HPOMethodParam("num_optimizer_steps", 3)
            ]
        },
        "hyper_parameters": [
            client.experiments.HPOParameter('learning_rate', min=0.0001, max=0.001, step=0.0003)
        ]
     }           

TRAINING_REFERENCESの定義で、前に定義した**TRAINING**の設定を参照します。

In [22]:
experiment_metadata = {
            client.repository.ExperimentMetaNames.NAME: "MNIST experiment",
            client.repository.ExperimentMetaNames.DESCRIPTION: "Best model for MNIST.",
            client.repository.ExperimentMetaNames.AUTHOR_EMAIL: "akaishi@jp.ibm.com",
            client.repository.ExperimentMetaNames.EVALUATION_METHOD: "multiclass",
            client.repository.ExperimentMetaNames.EVALUATION_METRICS: ["val_acc"],
            client.repository.ExperimentMetaNames.TRAINING_DATA_REFERENCE: TRAINING_DATA_REFERENCE,
            client.repository.ExperimentMetaNames.TRAINING_RESULTS_REFERENCE: TRAINING_RESULTS_REFERENCE,
            client.repository.ExperimentMetaNames.TRAINING_REFERENCES: [
                        {
                            "name": "MNIST_MLP",
                            "training_definition_url": definition_url,
                            "compute_configuration": {"name": "k80x2"},
                            "hyper_parameters_optimization": HPO
                        }],
            }

#### experimentをリポジトリに登録します

In [23]:
experiment_details = client.repository.store_experiment(meta_props=experiment_metadata)
experiment_uid = client.repository.get_experiment_uid(experiment_details)
print(experiment_uid)

9c28de3a-eaf0-4936-ac86-12c6b7756cd2


#### 登録済みexperimentsの一覧表示

In [24]:
client.repository.list_experiments()

------------------------------------  ----------------  ------------------------
GUID                                  NAME              CREATED
9c28de3a-eaf0-4936-ac86-12c6b7756cd2  MNIST experiment  2018-04-27T11:53:32.095Z
------------------------------------  ----------------  ------------------------


In [25]:
# 参考コード

# 既存のexperimentの更新
# updated_experiment_details = client.repository.update_experiment(experiment_uid, experiment_metadata)

# experiment定義の取得
# details = client.repository.get_experiment_details(experiment_uid)

# experiment定義の削除
# client.repository.delete(experiment_uid)

# experimentの実行

In [26]:
experiment_run_details = client.experiments.run(experiment_uid, asynchronous=True)

In [27]:
# experimentの状況確認
client.experiments.list_runs()

------------------------------------  ------------------------------------  -----------------  -------  --------------------
GUID (experiment)                     GUID (run)                            NAME (experiment)  STATE    CREATED
9c28de3a-eaf0-4936-ac86-12c6b7756cd2  78b8a8ce-80e9-4184-8ef3-b8f2cfa7049d  MNIST experiment   pending  2018-04-27T11:53:42Z
------------------------------------  ------------------------------------  -----------------  -------  --------------------


In [28]:
# experiment run_uidの取得
experiment_run_uid = client.experiments.get_run_uid(experiment_run_details)
print(experiment_run_uid)

78b8a8ce-80e9-4184-8ef3-b8f2cfa7049d


In [29]:
# training run内の個別テスト結果の取得
client.experiments.list_training_runs(experiment_run_uid)

------------------  ---------  -------  --------------------  --------  -----------
GUID (training)     NAME       STATE    SUBMITTED             FINISHED  PERFORMANCE
training-fEBoIVGmg  MNIST_MLP  pending  2018-04-27T11:53:44Z  -         -
------------------  ---------  -------  --------------------  --------  -----------


In [30]:
# 詳細結果の取得
experiment_run_details = client.experiments.get_run_details(experiment_run_uid)

In [31]:
# experiment実行状況の取得
client.experiments.get_status(experiment_run_uid)

{'current_at': '2018-04-27T11:53:42Z',
 'current_iteration': 1,
 'state': 'pending',
 'submitted_at': '2018-04-27T11:53:42Z'}

In [32]:
# 詳細状況の取得
experiment_details = client.experiments.get_details(experiment_uid)
print(json.dumps(experiment_details, indent=2))

{
  "resources": [
    {
      "entity": {
        "experiment_run_status": {
          "current_iteration": 1,
          "current_at": "2018-04-27T11:53:42Z",
          "submitted_at": "2018-04-27T11:53:42Z",
          "state": "pending"
        },
        "training_statuses": [
          {
            "metrics": [],
            "submitted_at": "2018-04-27T11:53:44Z",
            "state": "pending",
            "training_url": "/v3/models/training-fEBoIVGmg",
            "training_reference_name": "MNIST_MLP",
            "training_guid": "training-fEBoIVGmg",
            "current_at": "2018-04-27T11:53:45Z",
            "training_definition_url": "https://ibm-watson-ml.mybluemix.net/v3/ml_assets/training_definitions/8d712ccf-9f7c-4176-bebd-a7a60005b1fc"
          }
        ]
      },
      "experiment": {
        "evaluation_definition": {
          "method": "multiclass",
          "metrics": [
            {
              "name": "val_acc"
            }
          ]
        },
      

In [33]:
# training runs uidsの取得
experiment_run_details = client.experiments.get_run_details(experiment_run_uid)
training_run_uids = client.experiments.get_training_uids(experiment_run_details)

for i in training_run_uids:
    print(i)

training-fEBoIVGmg


# モニター機能

In [ ]:
# experiment runのモニター
client.experiments.monitor_logs(experiment_run_uid)



########################################################################

Monitor started for experiment run: 78b8a8ce-80e9-4184-8ef3-b8f2cfa7049d

########################################################################




--------------------------------------------------------
Log monitor started for training run: training-fEBoIVGmg
--------------------------------------------------------


2018-04-27 11:54:42.538642: I tensorflow/core/platform/cpu_feature_guard.cc:137] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA

2018-04-27 11:54:42.852983: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1105] Found device 0 with properties: 

name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235

pciBusID: 0000:83:00.0

totalMemory: 11.17GiB freeMemory: 11.10GiB

2018-04-27 11:54:43.098638: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1105] Found device 1 with properties: 

name: Tesla K80 major: 3 minor: 7 mem

In [ ]:
# statusの確認
client.experiments.get_status(experiment_run_uid)['state']

# 品質の確認

In [ ]:
!pip install cufflinks

In [ ]:
# metricsの取得
metrics = client.experiments.get_latest_metrics(experiment_run_uid)
all_metrics = client.experiments.get_metrics(experiment_run_uid)

In [ ]:
import sys
import pandas
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
import plotly.graph_objs as go

init_notebook_mode(connected=True)
sys.path.append("".join([os.environ["HOME"]])) 

In [ ]:
# Pandasでmetricsの表示
import pandas as pd

metrics_df = pd.DataFrame(columns=['GUID', 'NAME', 'METRIC NAME', 'METRIC VALUE'])

for m in metrics:
    for v in m['metrics']['values']:
            metrics_df = metrics_df.append({'GUID': m['training_guid'], 'NAME': m['training_reference_name'], 'METRIC NAME': v['name'], 'METRIC VALUE': v['value']}, ignore_index=True)
    
metrics_df

In [ ]:
# バーチャートでグラフ化
data = []
for i in list(pd.unique(metrics_df['METRIC NAME'])):
    data.append(go.Bar(x=metrics_df[metrics_df['METRIC NAME'].isin([i])]['GUID'] + ' (' + metrics_df[metrics_df['METRIC NAME'].isin([i])]['NAME'] + ')', y=metrics_df[metrics_df['METRIC NAME'].isin([i])]['METRIC VALUE'], name=i))
layout = go.Layout(
    barmode='group'
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)

# オンラインサービスのデプロイ

In [ ]:
status = client.experiments.get_status(experiment_run_uid)
best_model_uid = status['best_results']['experiment_best_model']['training_guid']
best_model_name = status['best_results']['experiment_best_model']['training_reference_name']

print(best_model_uid + ' (' +  best_model_name  + ')')

In [ ]:
saved_model_details = client.repository.store_model(best_model_uid, {'name': 'MNIST best model'})

In [ ]:
model_guid = client.repository.get_model_uid(saved_model_details)
print("Saved model guid: " + model_guid)

In [ ]:
client.repository.list_models()

In [ ]:
deployment_details = client.deployments.create(name="MNIST keras deployment", model_uid=model_guid)

In [ ]:
scoring_url = client.deployments.get_scoring_url(deployment_details)
print(scoring_url)

# 予測実行

In [ ]:
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
for i, image in enumerate([x_test[0], x_test[1]]):
    plt.subplot(2, 2, i + 1)
    plt.axis('off')
    plt.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')

In [ ]:
image_1 = x_test[0].ravel()
image_2 = x_test[1].ravel()
scoring_data = {'values': [image_1.tolist(), image_2.tolist()]}
predictions = client.deployments.score(scoring_url, scoring_data)
print("Scoring result: " + str(predictions))